In [72]:
import nltk
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import 	WordNetLemmatizer
from nltk.corpus import stopwords


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [73]:
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv("/SMS_train.csv", encoding='latin1')


**Data Preprocessing**

In [75]:
df

,S. No.,Message_body,Label
0,1,Rofl. Its true to its name,Non-Spam
1,2,The guy did some bitching but I acted like i'd...,Non-Spam
2,3,"Pity, * was in mood for that. So...any other s...",Non-Spam
3,4,Will ü b going to esplanade fr home?,Non-Spam
4,5,This is the 2nd time we have tried 2 contact u...,Spam
...,...,...,...
952,953,hows my favourite person today? r u workin har...,Non-Spam
953,954,How much you got for cleaning,Non-Spam
954,955,Sorry da. I gone mad so many pending works wha...,Non-Spam
955,956,Wat time ü finish?,Non-Spam


In [76]:
df.isnull().any()  # checking for null values if any

S. No.          False
Message_body    False
Label           False
dtype: bool

In [77]:
#text preprocessing
train_set = df['Message_body']


#remove stopwords and urls
def removing_stop_words(dataset):
    stop_words = set(stopwords.words('english'))
    filtered_sentence=[]
    for tweet in dataset:
        temp_tweet=' '.join([word for word in tweet.split() if word not in stop_words])
        filtered_sentence.append(temp_tweet)

    return filtered_sentence
train_set=removing_stop_words(df['Message_body'])
def removing_urls(dataset):
    cleaned_sentences=[]
    for tweet in dataset:
        url_re=re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet)
        cleaned_sentences.append(url_re)
    return cleaned_sentences
train_set=removing_urls(train_set)


In [78]:
train_set[1]

"The guy bitching I acted like i'd interested buying something else next week gave us free"

In [79]:
#removing punctuations
df['MB'] = df['Message_body'].str.replace("[^a-zA-Z0-9]", " ")
df = df[['Message_body','MB','Label']]
df.head()

<ipython-input-79-46cb6fdf4383>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['MB'] = df['Message_body'].str.replace("[^a-zA-Z0-9]", " ")


,Message_body,MB,Label
0,Rofl. Its true to its name,Rofl Its true to its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like i d...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that So any other s...,Non-Spam
3,Will ü b going to esplanade fr home?,Will b going to esplanade fr home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,Spam


In [80]:
#Replacing short words with space
df['MB'] = df['MB'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2]))
df.head()

<ipython-input-80-38cf220d97b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MB'] = df['MB'].apply(lambda row: ' '.join([word for word in row.split() if len(word)>2]))


,Message_body,MB,Label
0,Rofl. Its true to its name,Rofl Its true its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,The guy did some bitching but acted like inter...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",Pity was mood for that any other suggestions,Non-Spam
3,Will ü b going to esplanade fr home?,Will going esplanade home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,This the 2nd time have tried contact have won ...,Spam


In [81]:
#Changing sentences to LowerCase character
df['MB'] = [review.lower() for review in df['MB']]
df.head()


<ipython-input-81-879872c571b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MB'] = [review.lower() for review in df['MB']]


,Message_body,MB,Label
0,Rofl. Its true to its name,rofl its true its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,the guy did some bitching but acted like inter...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",pity was mood for that any other suggestions,Non-Spam
3,Will ü b going to esplanade fr home?,will going esplanade home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,this the 2nd time have tried contact have won ...,Spam


In [82]:
# lemmatization

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

def lemma(x):
    l=[]
    for i in x:
        l.append(lemmatizer.lemmatize(i))
    return l


In [83]:
df

,Message_body,MB,Label
0,Rofl. Its true to its name,rofl its true its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,the guy did some bitching but acted like inter...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",pity was mood for that any other suggestions,Non-Spam
3,Will ü b going to esplanade fr home?,will going esplanade home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,this the 2nd time have tried contact have won ...,Spam
...,...,...,...
952,hows my favourite person today? r u workin har...,hows favourite person today workin hard couldn...,Non-Spam
953,How much you got for cleaning,how much you got for cleaning,Non-Spam
954,Sorry da. I gone mad so many pending works wha...,sorry gone mad many pending works what,Non-Spam
955,Wat time ü finish?,wat time finish,Non-Spam


In [84]:
#stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
train_set = [ps.stem(word) for word in train_set if not word in set(stopwords.words('english'))]

In [85]:
df

,Message_body,MB,Label
0,Rofl. Its true to its name,rofl its true its name,Non-Spam
1,The guy did some bitching but I acted like i'd...,the guy did some bitching but acted like inter...,Non-Spam
2,"Pity, * was in mood for that. So...any other s...",pity was mood for that any other suggestions,Non-Spam
3,Will ü b going to esplanade fr home?,will going esplanade home,Non-Spam
4,This is the 2nd time we have tried 2 contact u...,this the 2nd time have tried contact have won ...,Spam
...,...,...,...
952,hows my favourite person today? r u workin har...,hows favourite person today workin hard couldn...,Non-Spam
953,How much you got for cleaning,how much you got for cleaning,Non-Spam
954,Sorry da. I gone mad so many pending works wha...,sorry gone mad many pending works what,Non-Spam
955,Wat time ü finish?,wat time finish,Non-Spam
